### File list Check

In [1]:
ls

HAT-L_SRx4_ImageNet-pretrain.pth  test/     train/     Untitled.ipynb
sample_submission.csv             test.csv  train.csv  upscale_train/


### Import library

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms.v2 as T
import torchvision.models as models
from torchsampler import ImbalancedDatasetSampler


from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import f1_score

from tqdm.notebook import tqdm
import timm

import warnings
warnings.filterwarnings(action='ignore') 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Define SAM Optimizer

In [2]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

### Set parameters

In [3]:
CFG = {
    'IMG_SIZE':256,
    'TEST_IMG_SIZE':196,
    'EPOCHS':30,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':8,
    'SEED':41,
    'WEIGHT_DECAY':0.0001
}

### Seed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Modifiy Image root name & split train and validation

In [5]:
def change_dir(text):
    return './data/hat_train_new'+text[7:-4]+'_HAT-L_SRx4_ImageNet-pretrain.png'

In [6]:
df = pd.read_csv('./train.csv')
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df['hat_img_path'] = df['img_path'].apply(change_dir)
train, val, _, _ = train_test_split(df, df['label'], test_size=0.1, stratify=df['label'], random_state=CFG['SEED'])

In [7]:
df.at[0,'hat_img_path']

'./data/hat_train_new/TRAIN_00000_HAT-L_SRx4_ImageNet-pretrain.png'

### Define custom dataset

In [8]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
    def __len__(self):
        return len(self.img_path_list)
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = Image.open(img_path)
        if self.transforms is not None:
            image = self.transforms(image)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def get_labels(self):
        return self.label_list

### Prepare data

In [9]:
trans = T.Compose([T.ToImage(),
                   T.ToDtype(torch.float32, scale=True),
                   T.RandomChoice([
                       T.Compose([
                        T.Resize((CFG['IMG_SIZE'],CFG['IMG_SIZE'])),
                        T.RandomResizedCrop(CFG['TEST_IMG_SIZE']),
                       ]),
                       T.Resize((CFG['TEST_IMG_SIZE'],CFG['TEST_IMG_SIZE']))
                   ],p=[0.9,0.1]),
                 T.RandomHorizontalFlip(0.5),
                 T.RandomVerticalFlip(0.5),
                 T.RandomRotation(22.5),
                 T.AugMix(),
                   T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ])
train_dataset = CustomDataset(train['hat_img_path'].values.tolist()+train['img_path'].values.tolist()+train['upscale_img_path'].values.tolist(),
                              train['label'].values.tolist()*3,
                              trans)
train_loader = DataLoader(train_dataset, 
                          batch_size = CFG['BATCH_SIZE'], 
                        sampler=ImbalancedDatasetSampler(train_dataset),
                          #shuffle=True, 
                          num_workers=4, 
                          pin_memory=True)

trans = T.Compose([T.ToImage(),
                   T.ToDtype(torch.float32, scale=True),
                   T.Resize((CFG['TEST_IMG_SIZE'],CFG['TEST_IMG_SIZE'])),
                  T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),])
                   
val_dataset = CustomDataset(val['hat_img_path'].values.tolist(),
                            val['label'].values.tolist(),
                            trans)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=4, pin_memory=True)

### Define baseModel

In [10]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.convnext_base(pretrained=True)
        #self.backbone = models.efficientnet_b4(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

### Define train function

In [11]:
def train_func(model, optimizer, train_loader, val_loader, scheduler, device, n=-1):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    
    best_score = 0
    best_model = None
    cutmix = T.CutMix(num_classes=25)
    mixup = T.MixUp(num_classes=25)
    cutmix_or_mixup = T.RandomChoice([cutmix, mixup])
    
    for epoch in tqdm(range(1, CFG['EPOCHS']+1)):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader),leave=False):
            imgs, labels = cutmix_or_mixup(imgs, labels)
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            #optimizer.step()
            
            ####
            optimizer.first_step(zero_grad=True)
            criterion(model(imgs),labels).backward()
            optimizer.second_step(zero_grad=True)

            ####
            train_loss.append(loss.item())


        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        if scheduler is not None:
            scheduler.step()
            
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 Score : [{_val_score:.5f}] LR : [{scheduler.get_last_lr()}]')
       
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            if(n==-1):
                torch.save({'model' : model.state_dict()},
                     f'./eva_ran_all/best_model({_val_score}).pt')
            else:
                torch.save({'model' : model.state_dict()},
                     f'./{n}/best_model({_val_score}).pt')
    model.to('cpu')
    return best_model
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader),leave=False):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score

### Load model using timm

In [12]:
model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True, num_classes=25)

<All keys matched successfully>

### Train

In [13]:
#randomsample + cutmix + mixup

base_optimizer = torch.optim.AdamW  # define an optimizer for the "sharpness-aware" update
optimizer = SAM(model.parameters(), base_optimizer, lr = CFG["LEARNING_RATE"],weight_decay=CFG["WEIGHT_DECAY"])
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer.base_optimizer, mode='max', factor=0.1, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer.base_optimizer, T_0=2, T_mult=2, eta_min=1e-8)
infer_model = train_func(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/5344 [00:00<?, ?it/s]

  0%|          | 0/198 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.58633] Val Loss : [0.17221] Val F1 Score : [0.97470] LR : [[5.005e-06]]


  0%|          | 0/5344 [00:00<?, ?it/s]

...

  0%|          | 0/5344 [00:00<?, ?it/s]

  0%|          | 0/198 [00:00<?, ?it/s]

Epoch [21], Train Loss : [0.95545] Val Loss : [0.08237] Val F1 Score : [0.98306] LR : [[5.979476158470562e-06]]


  0%|          | 0/5344 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Inference

In [28]:
test = pd.read_csv('./test.csv')
def change_dir(text):
    return './data/hat_test_new'+text[6:-4]+'_HAT-L_SRx4_ImageNet-pretrain.png'
    
test['hat_img_path'] = test['img_path'].apply(change_dir)
trans = T.Compose([T.ToImage(),
                   T.ToDtype(torch.float32,scale=True),
                   T.Resize((224,224)),
                   T.CenterCrop((196,196)),
                  T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                  ])
test_dataset = CustomDataset(test['hat_img_path'].values, None,trans)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE']*2, shuffle=False, num_workers=4)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

infer_model.to(device)
preds = inference(infer_model, test_loader, device)

  0%|          | 0/425 [00:00<?, ?it/s]

### Submit

In [27]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.to_csv('./no_hat.csv', index=False)